In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.inspection import permutation_importance
from pytorch_tabnet.tab_model import TabNetClassifier

# Load and clean the yeast dataset
data = pd.read_csv('yeast.csv').dropna()

print(data.head())

# Define the feature set and target variable
features = ['mcg', 'gvh', 'alm', 'mit', 'erl', 'pox', 'vac', 'nuc']
X = data[features]
y = data['name']

# Step 3: Convert categorical labels to numeric format
encoder = LabelEncoder()
y_numeric = encoder.fit_transform(y)

# Apply SMOTE for data augmentation
min_samples_per_class = min(np.bincount(y_numeric))
smote = SMOTE(random_state=42, k_neighbors=min(min_samples_per_class - 1, 5))
X_augmented, y_augmented = smote.fit_resample(X, y_numeric)

print(f"Original dataset size: {X.shape[0]}, Augmented size: {X_augmented.shape[0]}")

# Scale the features
scaler = StandardScaler()
X_augmented_scaled = scaler.fit_transform(X_augmented)

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_augmented_scaled, y_augmented, 
                                                      test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
rf = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [100, 200], 'max_depth': [None, 10, 20], 
              'min_samples_split': [2, 5]}
grid_search = GridSearchCV(rf, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_

# Train a Decision Tree model
dt = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=5)
dt.fit(X_train, y_train)

# Generate predictions for both models without filtering features
rf_train_preds = best_rf.predict(X_train)
dt_train_preds = dt.predict(X_train)
rf_valid_preds = best_rf.predict(X_valid)
dt_valid_preds = dt.predict(X_valid)

# Combine feature set with predictions for TabNet model
X_train_combined = np.column_stack((X_train, rf_train_preds, dt_train_preds))
X_valid_combined = np.column_stack((X_valid, rf_valid_preds, dt_valid_preds))

# Initialize and fit the TabNet model
tabnet = TabNetClassifier(n_d=8, n_a=8, n_steps=3, optimizer_params=dict(lr=0.02))
tabnet.fit(X_train_combined, y_train, max_epochs=50, patience=10, batch_size=32, virtual_batch_size=16)

# Generate predictions using TabNet
tabnet_preds = tabnet.predict(X_valid_combined)

# Evaluate models and display results
print("Random Forest Classification Report:")
print(classification_report(y_valid, rf_valid_preds, target_names=encoder.classes_))

print("Decision Tree Classification Report:")
print(classification_report(y_valid, dt_valid_preds, target_names=encoder.classes_))

print("TabNet Classification Report:")
print(classification_report(y_valid, tabnet_preds, target_names=encoder.classes_))

# Create an ensemble model (Voting Classifier)
ensemble = VotingClassifier(estimators=[('rf', best_rf), ('dt', dt)], voting='hard')
ensemble.fit(X_train, y_train)

# Evaluate the ensemble model
ensemble_preds = ensemble.predict(X_valid)
print("Ensemble Model Classification Report:")
print(classification_report(y_valid, ensemble_preds, target_names=encoder.classes_))

# Save models and preprocessing objects
with open('random_forest_model.pkl', 'wb') as rf_file:
    pickle.dump(best_rf, rf_file)

with open('decision_tree_model.pkl', 'wb') as dt_file:
    pickle.dump(dt, dt_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('label_encoder.pkl', 'wb') as encoder_file:
    pickle.dump(encoder, encoder_file)

tabnet.save_model('fine_tuned_tabnet')

print("All models have been trained, tuned, evaluated, and saved successfully.")


    mcg   gvh   alm   mit  erl  pox   vac   nuc name
0  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22  MIT
1  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22  MIT
2  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22  MIT
3  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22  NUC
4  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22  MIT
Original dataset size: 1484, Augmented size: 4630


C:\Users\Harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
C:\Users\Harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.47078 |  0:00:03s
epoch 1  | loss: 0.92295 |  0:00:08s
epoch 2  | loss: 0.65536 |  0:00:13s
epoch 3  | loss: 0.55768 |  0:00:17s
epoch 4  | loss: 0.55239 |  0:00:22s
epoch 5  | loss: 0.52657 |  0:00:27s
epoch 6  | loss: 0.46645 |  0:00:33s
epoch 7  | loss: 0.46356 |  0:00:36s
epoch 8  | loss: 0.43941 |  0:00:40s
epoch 9  | loss: 0.40999 |  0:00:42s
epoch 10 | loss: 0.32003 |  0:00:45s
epoch 11 | loss: 0.39634 |  0:00:48s
epoch 12 | loss: 0.30888 |  0:00:51s
epoch 13 | loss: 0.30662 |  0:00:54s
epoch 14 | loss: 0.28482 |  0:00:57s
epoch 15 | loss: 0.3212  |  0:00:59s
epoch 16 | loss: 0.22647 |  0:01:02s
epoch 17 | loss: 0.20133 |  0:01:05s
epoch 18 | loss: 0.33017 |  0:01:10s
epoch 19 | loss: 0.22924 |  0:01:15s
epoch 20 | loss: 0.26715 |  0:01:19s
epoch 21 | loss: 0.21651 |  0:01:22s
epoch 22 | loss: 0.19177 |  0:01:24s
epoch 23 | loss: 0.17803 |  0:01:27s
epoch 24 | loss: 0.19917 |  0:01:30s
epoch 25 | loss: 0.23245 |  0:01:33s
epoch 26 | loss: 0.19944 |  0:01:36s
e

C:\Users\Harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Harish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Ensemble Model Classification Report:
              precision    recall  f1-score   support

         CYT       0.49      0.62      0.55        91
         ERL       1.00      1.00      1.00        92
         EXC       0.91      0.90      0.91        80
         ME1       0.98      0.99      0.98        92
         ME2       0.92      0.96      0.94        93
         ME3       0.84      0.93      0.88       100
         MIT       0.66      0.73      0.69        92
         NUC       0.51      0.36      0.42        98
         POX       0.97      0.96      0.96        92
         VAC       0.96      0.79      0.87        96

    accuracy                           0.82       926
   macro avg       0.82      0.82      0.82       926
weighted avg       0.82      0.82      0.82       926

Successfully saved model at fine_tuned_tabnet.zip
All models have been trained, tuned, evaluated, and saved successfully.
